In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
from torch.optim import RMSprop
from datetime import datetime, timedelta

/Users/Yasser/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [7]:
def transformed_time(t,T):
    return 2*(t-T)/T+1

def transformed_inventory_action(q0,q,x):
    q_hat=q/q0-1
    x_hat=x/q0
    r=np.sqrt(q_hat**2+x_hat**2)
    theta=np.arctan(-x_hat/q_hat)
    chi=-x_hat/q_hat
    if theta<=np.pi/4:
        radial_dist=r*np.sqrt((chi**2+1)*(2*np.cos(np.pi/4-theta)**2))
    else:
        radial_dist=r*np.sqrt((chi**(-2)+1)*(2*np.cos(theta-np.pi/4)**2))
    q_transform=-radial_dist*np.cos(theta)
    x_transform=radial_dist*np.sin(theta)
    return q_transform,x_transform

def transformed_price(midprice_series):
    """
    Computes the transformed price feature (P̃) from a time series of midprices.

    Parameters:
        midprice_series (pd.Series): Series indexed by timestamp (datetime), with midprice per second.

    Returns:
        pd.Series: Transformed price series (P̃), same index as input.
    """
    # Ensure the series is sorted by time
    midprice_series = midprice_series.sort_index()

    # Group by hour
    grouped = midprice_series.groupby(midprice_series.index.floor('H'))

    transformed_series = []

    for hour, group in grouped:
        # Subtract opening price of the hour
        opening_price = group.iloc[0]
        centered = group - opening_price

        # Estimate scale to fit mostly within [-1, 1]
        lower, upper = np.percentile(centered, [1, 99])  # clip only outliers
        if lower!=upper:
            # Affine transformation
            transformed = (2/(upper-lower))*(centered-upper)+1
        else:
            transformed =0*centered
        
        transformed_series.append(transformed)

    return pd.concat(transformed_series)

def QV(midprice_series):
    return np.sum(midprice_series.diff()**2)



In [8]:
class TradingEnv:
    def __init__(self, start_date,T,N, delta_t,price_data, initial_inventory=500,a=1):
        """Time is expressed in second"""
        self.start_date=start_date
        self.T=T
        self.delta_t=delta_t #period between succesive trades in min 
        self.Tk_list = np.array([T/N*i for i in range(1,N)])
        self.Mk=T/N/delta_t
        self.price_data = price_data
        self.transormed_price=transformed_price(price_data)
        self.initial_inventory = initial_inventory

        self.current_period_index = 0
        self.inventory = initial_inventory
        self.time = self.Tk_list[0]
        self.done = False
        self.a=a

        self.state=self.get_state(self.time)

    def reset(self):
        self.current_period_index = 0
        self.inventory = self.initial_inventory
        self.time = self.Tk_list[0]
        self.done = False
        return self.get_state(self.time)

    def step(self, action):
            x_Tk = action
            # bornes de temps de la période courante
            t0 = self.start_date + timedelta(seconds=self.Tk_list[self.current_period_index])
            t1 = self.start_date + timedelta(seconds=self.Tk_list[self.current_period_index + 1])
            prices = self.price_data.loc[t0:t1]
            n = len(prices)

            # 1) reward de trade intra-période
            if n > 0:
                trade_reward = np.sum((self.inventory / n) * prices.diff())
            else:
                trade_reward = 0.0

            # 2) pénalité quadratique (on ne divise par Mk que si Mk>0)
            if self.Mk > 0:
                penalty = self.a * (x_Tk / self.Mk) ** 2
            else:
                penalty = 0.0

            reward = trade_reward - penalty

            # mise à jour de l'état
            self.inventory -= x_Tk
            self.current_period_index += 1
            if self.inventory <= 0 or self.current_period_index >= len(self.Tk_list) - 1:
                self.done = True

            next_t = self.Tk_list[min(self.current_period_index, len(self.Tk_list)-1)]
            next_state = self.get_state(next_t)
            return next_state, reward, self.done, {}
        
    def get_state(self, T_i):

        if self.current_period_index>0:
            mask=(self.start_date+timedelta(seconds=self.Tk_list[self.current_period_index-1])<=self.price_data.index)&(self.price_data.index<self.start_date+timedelta(seconds=self.Tk_list[self.current_period_index]))
        else:
            mask=(self.start_date<=self.price_data.index)&(self.price_data.index<self.start_date+timedelta(seconds=self.Tk_list[self.current_period_index]))
            
        selected_times=self.price_data.index[mask]
        prices=self.price_data.loc[selected_times]
        qv=QV(prices)

        state = [
            T_i,
            self.inventory,
            self.price_data.loc[self.start_date+timedelta(seconds=T_i):].values[0],
            qv,
        ]
        return np.array(state, dtype=np.float32)



In [3]:
class QNetwork(nn.Module):
    def __init__(self, input_dim):
        super(QNetwork, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(20, 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(20, 20),
            nn.ReLU(),

            nn.Linear(20, 20),
            nn.ReLU(),

            nn.Linear(20, 20),
            nn.ReLU(),

            nn.Linear(20, 1)  # Q-value output
        )

    def forward(self, x):
        return self.net(x)



class TradingAgentRL:
    def __init__(self, env,state_dim, epsilon=0.1, tau=0.995, gamma=0.99, batch_size=5, memory_capacity=100, update_target_freq=10, lr=1e-3):
        self.env=env
        self.state_dim = state_dim
        self.epsilon = epsilon
        self.tau = tau
        self.gamma = gamma
        self.batch_size = batch_size
        self.update_target_freq = update_target_freq

        self.memory = deque(maxlen=memory_capacity)
        self.Q_main = QNetwork(state_dim)
        self.Q_target = QNetwork(state_dim)
        self.Q_target.load_state_dict(self.Q_main.state_dict())
        self.optimizer = RMSprop(self.Q_main.parameters(), lr)
        self.loss_fn = nn.MSELoss()

        self.iteration = 0

    def choose_action(self, state):
        q_i = state[1] 
        T_i=state[0]

        ## I we have reached terminal period [TN-1,T], we sell all the inventory 
        if T_i>=self.env.Tk_list[-1]:
            action=q_i
        else:
            if np.random.rand() < self.epsilon:
                action = np.random.binomial(q_i,1/(self.env.T-T_i))
            else:
                with torch.no_grad():
                    state_tensor = torch.FloatTensor(state).unsqueeze(0).repeat(int(q_i)+1,1)
                    actions_tensor = torch.arange(0, int(q_i) + 1).float()
                    inputs = torch.cat([state_tensor, actions_tensor.unsqueeze(1)], dim=1)
                    q_values = self.Q_main(inputs).squeeze()
                    action = torch.argmax(q_values).item()

        return action
    

    def store_transition(self, state, action, reward, next_state):
        self.memory.append((state, action, reward, next_state))

    def train_step(self):

        # We can update Q only if we have seen enough experience (state,action,reward,next_state)
        if len(self.memory) < self.batch_size:
            return

        # We sample batch_size trasnitions from memory
        minibatch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states = zip(*minibatch)

        states = torch.FloatTensor(states)
        actions = torch.FloatTensor(actions)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(next_states)

        # Target computation
        with torch.no_grad():

            next_q_values=[]
            for j in range(self.batch_size):

                q_range = int(next_states[j][1])  # inventory
                ns_batch = next_states[j].unsqueeze(0).repeat(q_range + 1, 1)
                actions_batch = torch.arange(0, q_range + 1).float()
                inputs = torch.cat([ns_batch, actions_batch.unsqueeze(1)], dim=1)

                if next_states[j][0]==self.env.T:
                    next_q_value = 0
                elif next_states[j][0]==self.env.Tk_list[-1]:
                    #R(s,q)=q(p′ −p)−aq2,
                    q=states[j][1]
                    T=self.env.start_date+timedelta(seconds=self.env.T)
                    delta=self.env.delta_t
                    prices=self.env.price_data 
                    p=prices.loc[T:].values[0]
                    p_prim=prices.loc[T+timedelta(seconds=delta):].values[0]
                    next_q_value = q*(p_prim-p)-self.env.a*q**2#f(next_states[j],states[j][1])
                else:
                    

                    #Compute Q-values with main network
                    self.Q_main.eval()
                    q_values = self.Q_main(inputs).squeeze()
                    ns=torch.tensor(next_states[j])
                    # Select action with highest Q-value
                    action = torch.tensor([torch.argmax(q_values).item()])
                    #Compute future Q-value with target network
                    input_target = torch.cat([ns, action])  # shape: [5]
                    self.Q_target.eval()
                    next_q_value = self.Q_target(input_target.unsqueeze(0)).item()

                next_q_values.append(next_q_value)        
            
            next_q_values = torch.FloatTensor(next_q_values)


        targets=rewards+self.gamma*next_q_values
        inputs = torch.cat([states ,actions.unsqueeze(1)], dim=1)
        q_preds = self.Q_main(inputs)
        loss = self.loss_fn(q_preds.squeeze(), targets)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.iteration += 1

    def train(self, num_episodes=100, N=100, update_target_every=10, tau=0.995):
        """
        Entraîne l'agent sur l'environnement `env` pendant `num_episodes` épisodes.
        À chaque épisode, effectue N pas d'interaction.
        """
        last_date=self.env.price_data.index[-1]
        dates=self.env.price_data.loc[:last_date-timedelta(seconds=self.env.T)].index
        for episode in range(num_episodes):
            # One episode is defined as an execution period of lenght T, chosen randomly in the dataset (2018 to 2023)
            self.env.start_date=random.choice(dates)
            # State is reset at the beginning of each episode
            state = self.env.reset()  
            for i in range(N): # N is the number of period T0<T1..<TN-1 such that an action is taken at each T_i

                #choose action according to epsilon-greedy policy
                action =self.choose_action(state)

                # Update current state of the environment 
                next_state, reward, done, _ = self.env.step(action)
                
                # Save transition for experience replay
                self.store_transition(state, action, reward, next_state)

                state=next_state

                # Update Q with experience replay
                self.train_step()

                if done:
                    break

            # Mise à jour du réseau cible
            if episode % update_target_every == 0:
                self.Q_target.load_state_dict(self.Q_main.state_dict())

            # Décroissance de ε
            self.epsilon = max(self.epsilon * tau, 0.01)

            print(f"Episode {episode+1}/{num_episodes} terminé, ε = {self.epsilon:.4f}")



In [9]:
data=pd.read_csv("Data/BTC_ETH_15mn.csv")
data.Date=data.Date.apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
data=data.set_index("Date")

In [10]:
data

,Price,Volume,Price (ETH),Volume (ETH)
Date,,,,
2018-10-24 00:00:00,6476.540000,3.708780e+09,204.327000,1232950000
2018-10-24 00:15:00,6483.150000,3.709670e+09,204.378000,1236570000
2018-10-24 00:30:00,6474.200000,3.677130e+09,204.439000,1239840000
2018-10-24 00:45:00,6486.190000,3.661460e+09,204.441000,1237220000
2018-10-24 01:00:00,6480.084479,3.641914e+09,204.070635,1227899090
...,...,...,...,...
2023-11-27 08:45:00,37346.223298,1.628826e+10,2046.790071,9322101316
2023-11-27 09:00:00,37401.430405,1.635176e+10,2049.064393,9328586333
2023-11-27 09:15:00,37405.996546,1.648487e+10,2049.523629,9353842431


In [11]:
env=TradingEnv(data.Price.index[0],15*500*60,100,15*60,data.Price)
agent=TradingAgentRL(env,5)

In [12]:

agent.train( num_episodes=2, N=70, update_target_every=10, tau=0.995)

<ipython-input-3-e62f627eef0b>:85: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  states = torch.FloatTensor(states)
<ipython-input-3-e62f627eef0b>:118: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ns=torch.tensor(next_states[j])


Episode 1/2 terminé, ε = 0.0995
Episode 2/2 terminé, ε = 0.0990


In [49]:
def pnl_agent(price_series, start_date, actions, delta_t, a, initial_inventory):
    """
    Eq (3.1)-(3.2) :
      R_{k,i} = q_{t_{k,i}}*(p_{i+1}-p_i) - a*(x_k/M_k)^2
      P&L = sum_{k,i} R_{k,i}  (avec q_{t_{k,0}} = q0 au début de chaque k)
    """
    pnl = 0.0
    q = initial_inventory
    p0 = price_series.loc[start_date]
    pnl -= q * p0  # terme -q0*p0 parfois inclus dans la forme développée (3.4)

    for Tk, xk in actions:
        # récupérer les prix de Tk à Tk+1
        block = price_series.loc[Tk : Tk + timedelta(seconds=delta_t)]
        prices = block.values
        M = len(prices) - 1
        if M <= 0:
            continue

        v = xk / M  # quantité par sub-tick
        for i in range(M):
            dp = prices[i+1] - prices[i]
            pnl += q * dp            # q_{t_{k,i}} * (p_{i+1}-p_i)
            pnl -= a * (v ** 2)      # pénalité quadratique
            q  -= v                  # on diminue q au fil des sous-ticks

    return pnl

In [50]:
def pnl_twap(price_series, start_date, Q0, Tk_list, delta_t, a):
    """
    Eq (7.1): x_k = Q0/N
    Puis on appelle pnl_agent avec ce plan d'actions.
    """
    N = len(Tk_list) - 1
    x_per = Q0 / N
    actions = [
        (start_date + timedelta(seconds=int(Tk_list[k])), x_per)
        for k in range(N)
    ]
    return pnl_agent(price_series, start_date, actions, delta_t, a, Q0)



In [33]:
from datetime import timedelta
import pandas as pd
min_time = data.index.min()
max_time = data.index.max() - timedelta(seconds=env.T)
# horaire plein toutes les T secondes
test_dates = pd.date_range(min_time, max_time, freq=f"{env.T}s")


In [52]:
def evaluate(agent, start_dates):
    """
    Pure greedy evaluation (ε=0) : compare P&L_RL vs P&L_TWAP en bps.
    Renvoie mean, median, glr, prob.
    """
    orig_eps = agent.epsilon
    agent.epsilon = 0.0

    deltas = []
    for sd in start_dates:
        # tronquer la série à [sd, sd+T]
        series = agent.env.price_data.loc[
            sd : sd + timedelta(seconds=agent.env.T)
        ]

        # reset env
        agent.env.start_date = sd
        state = agent.env.reset()

        # collecter actions greedy
        actions = []
        for Tk in agent.env.Tk_list[:-1]:
            x = agent.choose_action(state)
            t_dt = sd + timedelta(seconds=int(Tk))
            actions.append((t_dt, x))
            state, _, done, _ = agent.env.step(x)
            if done: break

        # calcul des P&L
        pnl_rl  = pnl_agent(
            series, sd, actions,
            agent.env.delta_t,
            agent.env.a,
            agent.env.initial_inventory
        )
        pnl_ref = pnl_twap(
            series, sd,
            agent.env.initial_inventory,
            agent.env.Tk_list,
            agent.env.delta_t,
            agent.env.a
        )

        if pnl_ref != 0:
            deltas.append(1e4 * (pnl_rl - pnl_ref) / pnl_ref)

    agent.epsilon = orig_eps

    arr = np.array(deltas, dtype=float)
    return {
        "mean"  : round(arr.mean(),    3),
        "median": round(np.median(arr),3),
        "glr"   : round(arr[arr>0].mean() / (-arr[arr<0].mean() + 1e-8), 3),
        "prob"  : round(float((arr>0).mean() * 100), 3),
    }

In [53]:
stats = evaluate(agent, test_dates)
print(stats)

{'mean': 393.279, 'median': 295.783, 'glr': 3.848, 'prob': 88.515}
